In [1]:
from scipy.integrate import odeint
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output, State
import plotly.graph_objs as go
%matplotlib inline

In [2]:
def deriv(y, t, N, beta, gamma, delta):
    S, E, I, R = y
    dSdt = -beta * S * I / N
    dEdt = beta * S * I / N - delta * E
    dIdt = delta * E - gamma * I
    dRdt = gamma * I
    return dSdt, dEdt, dIdt, dRdt

In [3]:
app = dash.Dash(__name__)

app.layout = html.Div([
    html.Img(id='corona_png',src='assets/covid-19.png'),
    html.Img(id='um6p_png',src='assets/um6p.png'),
    html.Div([
        html.Div([
            html.Label(children='Nomber of Susceptibles :'),
            dcc.Input(
                id='susceptibles',
                type='number',
                value=999
            ),

            html.Label(children='Nomber des infecteds :'),
            dcc.Input(
                id='infecteds',
                type='number',
                value=1
            ),

            html.Label(children='Recovered :'),
            dcc.Input(
                id = 'recovered',
                type='number',
                value=0
            ),

            html.Label(children='Days :'),
            dcc.Input(
                id = 'days',
                type='number',
                value=100
            ),
            
            html.Label(children='Number of susceptibles could be infect by an infected (Probabilty of an susceptible becomes exposed):'),
            dcc.Slider(
                id='exposed',
                min=0,
                max=10,
                value=1,
                marks = {str(round(val,2)) : str(round(val,2)) for val in np.arange(0,11)},
                step=None
            ),

            html.Label(children='Number of exposeds become infected (Probabilty of an exposeds becomes infected):'),
            dcc.Slider(
                id='infected',
                min=0,
                max=10,
                value=1,
                marks = {str(round(val,2)) : str(round(val,2)) for val in np.arange(0,11)},
                step=None
            ),


            html.Label(children='period of infections:'),
            dcc.Slider(
                id='infection',
                min=0,
                max=10,
                value=4,
                marks = {str(round(val,2)) : str(round(val,2)) for val in np.arange(0,11)},
                step=None
            ),


            html.Label(children='period of incubation:'),
            dcc.Slider(
                id='incubation',
                min=0,
                max=10,
                value=3,
                marks = {str(round(val,2)) : str(round(val,2)) for val in np.arange(0,11)},
                step=None
            ),
            
            html.Button(id='submit-button', n_clicks=0, children='Submit')

        ],id='form'),

        dcc.Graph(id='indicator-graphic'),
    ],id="resultat")
    
],className = 'container')


@app.callback(
    Output('indicator-graphic', 'figure'),
    [Input('submit-button', 'n_clicks')],
    [State('susceptibles', 'value'),
    State('infecteds', 'value'),
    State('recovered', 'value'),
    State('days', 'value'),
    State('exposed', 'value'),
    State('infected', 'value'),
    State('infection', 'value'),
    State('incubation', 'value')]
)
def update_figure(n_clicks,susceptibles,infecteds,recovered,days,exposed,infected,infection,incubation):
    N = susceptibles+infecteds+infecteds
    beta = exposed
    D = infection
    gamma = infected / D
    delta = infected / incubation  #à verfier

    S0, E0, I0, R0 = susceptibles, 0, infecteds, 0  
    t = np.linspace(0, days, 100)

    y0 = S0, E0, I0, R0 

    # Integrate the SIR equations over the time grid, t.
    ret = odeint(deriv, y0, t, args=(N, beta, gamma, delta))
    S, E, I, R = ret.T
    
    traces = [
            go.Scatter(x=[0, 0], y=[0, 0]),
            go.Scatter(x=[0, 0], y=[0, 0]),
            go.Scatter(x=[0, 0], y=[0, 0]),
            go.Scatter(x=[0, 0], y=[0, 0])
    ]
    
    layout = go.Layout(title = "SEIR Model",
                    xaxis=dict(range=[0, 100], autorange=False,title='Days'),
                    yaxis=dict(range=[0, 1000], autorange=False,title='Cases'),
                    updatemenus=[dict(
                        type="buttons",
                        buttons=[dict(label="Play",
                              method="animate",
                              args=[None,{"frame": {"duration": 4000, "redraw": False}, "transition": {"duration": 300000,
                                                                    "easing": "quadratic-in-out"}}])])]
                    )
    
    frames = [go.Frame(
        data=[go.Scatter(
            x = t,
            y = ax[0],
            mode = 'lines',
            name= ax[1]
        )for ax in [(S,'Susceptible'), (E,'Exposed'), (I,'Infected'), (R,'Recovered')]]
    )]
    
    return {
        'data' : traces,
        'layout' : layout,
        'frames' :frames
    }

In [ ]:
if __name__ == '__main__':
    app.run_server(debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [21/Oct/2020 01:15:12] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Oct/2020 01:15:13] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [21/Oct/2020 01:15:13] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [21/Oct/2020 01:15:15] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Oct/2020 01:15:37] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Oct/2020 01:16:07] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Oct/2020 01:16:08] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [21/Oct/2020 01:16:08] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [21/Oct/2020 01:16:08] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Oct/2020 01:16:25] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Oct/2020 01:16:25] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [21/Oct/2020 01:16:25] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [21/Oct/2020 01:16:25] "GET /_favicon.ico?v=1.1